Análisis REM8
 ==========

Comienzo cargando todos los paquetes necesarios

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy 
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import datetime as dt
import datetime
from datetime import timedelta  
import timeboard as tb # turnos 

# para tratar de poner espanol
from datetime import date, datetime, time
from babel.dates import format_date, format_datetime, format_time, format_timedelta, Locale
import locale                                    # para tratar de poner espanol
#locale = Locale('es', 'CL')
locale.setlocale(locale.LC_ALL,'es_ES.UTF-8')  

import os
import runpy
from pyexcel_ods import get_data
from matplotlib_venn import venn3
from matplotlib_venn import venn2, venn2_circles
import math

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline


#otro graficador
import seaborn as sns

#stats

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

/home/egidio/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# cargo algunos scripts que me suelen servir para análisis

In [3]:
#listo los archivos
scripts = []
for dirname, dirnames, filenames in os.walk('/home/egidio/Dropbox/Compucosas/programas py'):
    for filename in filenames:
        scripts.append(os.path.join(dirname, filename))

scripts

# los cargo
for script in scripts:
    if script[-3:] == '.py':
        exec(open(str(script)).read())


['/home/egidio/Dropbox/Compucosas/programas py/lector_ODF.py',
 '/home/egidio/Dropbox/Compucosas/programas py/como importar',
 '/home/egidio/Dropbox/Compucosas/programas py/descripcion_univariada.py',
 '/home/egidio/Dropbox/Compucosas/programas py/características_operativas_de_test.py',
 '/home/egidio/Dropbox/Compucosas/programas py/percentiles.py',
 '/home/egidio/Dropbox/Compucosas/programas py/tabala_frecuencias.py',
 '/home/egidio/Dropbox/Compucosas/programas py/__pycache__/percentiles.cpython-36.pyc',
 '/home/egidio/Dropbox/Compucosas/programas py/__pycache__/lector_ODF.cpython-36.pyc',
 '/home/egidio/Dropbox/Compucosas/programas py/__pycache__/descripcion_univariada.cpython-36.pyc',
 '/home/egidio/Dropbox/Compucosas/programas py/__pycache__/características_operativas_de_test.cpython-36.pyc',
 '/home/egidio/Dropbox/Compucosas/programas py/__pycache__/tabala_frecuencias.cpython-36.pyc',
 '/home/egidio/Dropbox/Compucosas/programas py/__pycache__/funciones_locales.cpython-36.pyc']

# Cargo la BD

In [4]:
BD = pd.read_pickle('./BD/eSAMU.pkl')

In [5]:
BD.columns
BD.head(2)

Index(['Id', 'Fecha', 'Motivo del Llamado', 'Submotivo del Llamado',
       'Prioridad', 'Estado', 'Edad', 'Género', 'Dirección', 'Sector',
       'Comuna', 'Región', 'Intervención', 'Tipo Traslado',
       'Categoría Vehículo', 'Condición Traslado', 'En Regulación',
       'En Despacho', 'En Curso', 'Atendida', 'Completado', 'Regulado',
       'Usuario que Cierra', 'Móvil Agregado', 'Móvil Despachado',
       'Móvil Salida', 'Móvil Llegada Lugar', 'Móvil en Traslado',
       'Móvil Llegada a CA', 'Móvil Recepción Paciente',
       'Móvil Camilla Retenida', 'Móvil Vuelta a Base', 'Móvil LLegada a Base',
       'Móvil Finalizado', 'Nombre Vehículo', 'Tipo Requerido',
       'Tipo Despachado', 'Km Recorrido', 'Chofer', 'Paramédico', 'Enfermero',
       'Doctor', 'Longitud', 'Latitud', 'Diff_recep_Despachado',
       'Diff_Despacho_Salida', 'Diff_salida_enellugar', 'Diff_z6_1',
       'Diff_z6_2', 'Diff_z6_3'],
      dtype='object')

,Id,Fecha,Motivo del Llamado,Submotivo del Llamado,Prioridad,Estado,Edad,Género,Dirección,Sector,...,Enfermero,Doctor,Longitud,Latitud,Diff_recep_Despachado,Diff_Despacho_Salida,Diff_salida_enellugar,Diff_z6_1,Diff_z6_2,Diff_z6_3
0,131301,2020-01-31 23:58:13,Accidentes y Traumas,Caídas leves,S2,Completado,40.0,Femenino,arlegui con quillota,centro,...,Karen Urra coloma,NaN,NaN,NaN,03:03:22,00:01:11,00:03:37,NaT,NaT,NaT
1,131300,2020-01-31 23:29:08,Consciencia,Comportamientos extraños,NaN,Completado,63.0,Femenino,estacion metro,c9,...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT


para efectos de este análisis es importante dividir ambulancias según su complejidad

In [6]:
BD['Bas_avan'] = BD['Tipo Despachado'] 
BD.loc[BD['Tipo Despachado'] == 'm1' , 'Bas_avan' ] = 'Básico'
BD.loc[BD['Tipo Despachado'] == 'm2' , 'Bas_avan' ] = 'Avanzado'
BD.loc[BD['Tipo Despachado'] == 'm3' , 'Bas_avan' ] = 'Avanzado'
BD.loc[BD['Tipo Despachado'] == 'x5' , 'Bas_avan' ] = 'Avanzado'

defino Bas_avan como categórico

# defino la BD con la que trabajaré

El informe a continuación es un análisis del:
* fecha
* sector

## según fecha

In [7]:
rango = (                                            #mes Enero
    (BD.Fecha > pd.to_datetime('2020-02-01')) &
    (BD.Fecha < pd.to_datetime('2020-02-29'))
)


db1 = BD[rango].copy()
np.sort(db1.Fecha.dt.to_period('M').unique())

array([Period('2020-02', 'M')], dtype=object)

## segun ambulancias

In [8]:
db1['Nombre Vehículo'].unique()

array(['AP 14', 'R22', nan, 'R13', 'R11', 'R63', 'R12', 'R33', 'R20',
       'R71', 'R24', 'R31', 'AP16', 'R21', 'R92', 'R43', 'Móvil Ventana',
       'R23', 'R50', 'R51', 'R80', 'R61', 'AP51', 'R83', 'R41', 'R62',
       'R32', 'AP28', 'AP1', 'R52', 'Móvil Artificio', 'R25', 'R100',
       'AP15', 'Móvil Los Molles', 'Móvil Puchuncaví', 'R57', 'AP31',
       'AP41', 'REMA  Poli Naval Villa Alemana', 'R73', 'AT51',
       'MOVIL ALGARROBO', 'R91', 'R53', 'MOVIL LONCURA', 'R35',
       'Móvil Hijuelas', 'R10', 'AP27', 'AP26', 'Móvil Papudo',
       'MOVIL CATAPILCO', 'R42', 'R68', 'R84', 'Móvil Nogales', 'R72',
       'AT90', 'MOVIL SAPU VENTANA', 'W56', 'R102', 'AT53',
       'Móvil Loncura', 'R81', 'Móvil Rutas Abertis.', 'sapu zapallar',
       'MOVIL EL MELON', 'Móvil Las Parcelas', 'MOVIL PAPUDO',
       'MOVIL RUTA', 'nogales movil 8', 'R82'], dtype=object)

elegir uno

In [10]:
#elegir uno

# sector = db1['Nombre Vehículo'].str.contains('R3') == True
# db1 = db1[sector]

In [11]:
texto1 = 'Este es un inforque que considera el mes de '
texto2 = 'En el territorio de '

sector = 'SSVQ'
# sector = db1.Comuna.value_counts().index[0]

mes= db1.Fecha.describe()[4].strftime('%B')

result = texto1 + mes + '. ' + texto2 + sector + '.' + '\n \n'
result
print(result, file=open('Informe.txt','a'))

'Este es un inforque que considera el mes de febrero. En el territorio de SSVQ.\n \n'

In [12]:
db1.Comuna.value_counts()

Viña del Mar     1246
Quilpué           463
Quillota          334
Villa Alemana     295
Quintero          234
La Ligua          201
Limache           160
Concón            155
Calera            125
Puchuncaví         46
Cabildo            46
Nogales            39
Valparaíso         36
La Cruz            35
Olmué              28
Petorca            20
Hijuelas           12
Papudo             11
Zapallar            9
Algarrobo           4
Calle Larga         1
San Felipe          1
Casablanca          1
Name: Comuna, dtype: int64

# defino el archivo donde guardaré la info

Será un archivo

como dividir por fecha

In [13]:
np.sort(db1.Fecha.dt.to_period('M').unique())

array([Period('2020-02', 'M')], dtype=object)

## Sección K: INTERVENCIONES PRE HOSPITALARIAS (SAMU)

prestaciones según recurso

In [14]:
# los tiempos de z17 al z8 los divido según requisito planilla
cortado = pd.cut(
    db1.Diff_salida_enellugar.dt.seconds,
    [0, 20, 40, 1000000],
    labels = ['Menos 20 min',  '20 a 40 min',  'Más 40 min'])

# tabulo
#pd.crosstab(db1.Bas_avan, cortado, margins=True, margins_name= 'Total')

# tabulo por mes
print('Por Mes')
dum1 = cortado
for fecha in np.sort(db1.Fecha.dt.to_period('M').unique()):
    fecha
    test = db1.loc[db1.Fecha.dt.to_period('M') == fecha]
    result = pd.crosstab(
        test['Bas_avan'],
        dum1,
        margins=True, margins_name= 'Total', dropna=False)
    result.columns.name = 'Tiempo llegada al lugar'
    result.index.name = 'Tipo Móvil'
    result

Por Mes


Period('2020-02', 'M')

Tiempo llegada al lugar,Menos 20 min,20 a 40 min,Más 40 min,Total
Tipo Móvil,,,,
Avanzado,14,2,705,1223
Básico,44,2,732,1670
Total,64,4,1584,4560


escribo en el archivo

In [15]:
titulo = 'Sección K: INTERVENCIONES PRE HOSPITALARIAS (SAMU)'
texto = 'Se asume que todas las intervenciones críticas son realizadas \
por móviles avanzados y que las no-críticas por móviles básicos. Esto \
determina que las casillas de las prestaciones críticas tengan valores \
de cero para móviles básicos y las prestaciones no-críticas tengan \
valores cero para móviles avanzados. \n \n'
espacios = '\n \n'

titulo
texto
result

print(titulo, file=open('Informe.txt','a'))
print(texto, file=open('Informe.txt','a'))
print(result, file=open('Informe.txt','a'))
print(espacios, file=open('Informe.txt','a'))

'Sección K: INTERVENCIONES PRE HOSPITALARIAS (SAMU)'

'Se asume que todas las intervenciones críticas son realizadas por móviles avanzados y que las no-críticas por móviles básicos. Esto determina que las casillas de las prestaciones críticas tengan valores de cero para móviles básicos y las prestaciones no-críticas tengan valores cero para móviles avanzados. \n \n'

Tiempo llegada al lugar,Menos 20 min,20 a 40 min,Más 40 min,Total
Tipo Móvil,,,,
Avanzado,14,2,705,1223
Básico,44,2,732,1670
Total,64,4,1584,4560


## Sección L:  TRASLADOS PRIMARIOS A UNIDADES DE URGENCIA (Desde el lugar del evento a unidad de Emergencia)

In [16]:
db1['Categoría Vehículo'].value_counts()
#pd.crosstab(db1['Categoría Vehículo'], test['Bas_avan'])

db1['Categoría Vehículo'] = pd.Categorical(db1['Categoría Vehículo'],
                                           categories=['samu', 'enrutado', 'nosamu'])

samu        1284
enrutado     714
nosamu        22
Name: Categoría Vehículo, dtype: int64

In [17]:
for fecha in np.sort(db1.Fecha.dt.to_period('M').unique()):
    fecha
    test = db1.loc[db1.Fecha.dt.to_period('M') == fecha]
    test = test.loc[test['Motivo del Llamado'] != 'Traslados']
    result = test.groupby(['Categoría Vehículo', 'Bas_avan' ])['Id'].count()
    result.index.names = ['Categoría Vehículo', 'Tipo móvil']
    result

Period('2020-02', 'M')

Categoría Vehículo  Tipo móvil
samu                Avanzado      386
                    Básico        386
enrutado            Avanzado      113
                    Básico        253
nosamu              Avanzado        3
                    Básico         17
Name: Id, dtype: int64

escribo en el archivo

In [18]:
titulo = ' Sección L:  TRASLADOS PRIMARIOS A UNIDADES DE URGENCIA'
texto = ' '

titulo
texto
result

print(titulo, file=open('Informe.txt','a'))
print(texto, file=open('Informe.txt','a'))
print(result, file=open('Informe.txt','a'))
print(espacios, file=open('Informe.txt','a'))

' Sección L:  TRASLADOS PRIMARIOS A UNIDADES DE URGENCIA'

' '

Categoría Vehículo  Tipo móvil
samu                Avanzado      386
                    Básico        386
enrutado            Avanzado      113
                    Básico        253
nosamu              Avanzado        3
                    Básico         17
Name: Id, dtype: int64

## Sección M: TRASLADO SECUNDARIO (Desde un Establecimiento a Otro)

In [19]:
for fecha in np.sort(db1.Fecha.dt.to_period('M').unique()):
    fecha
    test = db1.loc[db1.Fecha.dt.to_period('M') == fecha]
    test = test.loc[test['Motivo del Llamado'] == 'Traslados']
    result = test.groupby(['Categoría Vehículo', 'Bas_avan' ])['Id'].count()
    result.index.names = ['Categoría Vehículo', 'Tipo móvil']
    result

Period('2020-02', 'M')

Categoría Vehículo  Tipo móvil
samu                Avanzado      198
                    Básico        214
enrutado            Avanzado       75
                    Básico        188
Name: Id, dtype: int64

In [20]:
titulo = ' Sección M: TRASLADO SECUNDARIO (Desde un Establecimiento a Otro)'
texto = ' '

titulo
texto
result

print(titulo, file=open('Informe.txt','a'))
print(texto, file=open('Informe.txt','a'))
print(result, file=open('Informe.txt','a'))
print(espacios, file=open('Informe.txt','a'))

' Sección M: TRASLADO SECUNDARIO (Desde un Establecimiento a Otro)'

' '

Categoría Vehículo  Tipo móvil
samu                Avanzado      198
                    Básico        214
enrutado            Avanzado       75
                    Básico        188
Name: Id, dtype: int64

# Sección N: CLASIFICACION DE LAS INTERVENCIONES POR GRANDES GRUPOS DE DIAGNOSTICOS (SAMU)

In [21]:
db1['Patol'] = np.nan
db1.loc[db1['Submotivo del Llamado'] == 'Accidente vehicular o transporte. Atropellos',
        'Patol'] = 'PoliTMT'
db1.loc[db1['Submotivo del Llamado'] == 'Accidente múltiples víctimas (cualquier razón)',
        'Patol'] = 'PoliTMT'
db1.loc[db1['Submotivo del Llamado'] == 'Caídas graves',
        'Patol'] = 'PoliTMT'

db1.loc[db1['Submotivo del Llamado'] == 'Colapso respiratorio o circulatorio. PCR. Asfixia',
        'Patol'] = 'PCR'

db1.loc[db1['Submotivo del Llamado'] == 'Dolor de pecho',
        'Patol'] = 'SCA'

db1.loc[db1['Patol'].isnull(),
        'Patol'] = 'Otros'

db1['Patol'] = pd.Categorical(db1['Patol'], categories=['SCA', 'PCR', 'PoliTMT', 'Otros'])

In [22]:
db1.loc[db1['Género'] == 'Femenino', 'Género'] = 'Feme'
db1.loc[db1['Género'] == 'Masculino', 'Género'] = 'Masc'

db1['Género'] = pd.Categorical(db1['Género'], categories=['Masc', 'Feme'])

In [23]:
for fecha in np.sort(db1.Fecha.dt.to_period('M').unique()):
    fecha
    test = db1.loc[db1.Fecha.dt.to_period('M') == fecha]
    dum1 = pd.cut(test.Edad, range(0, 90, 5), right=False)  
    test['Patol'].value_counts(sort= False)
    result0 = pd.crosstab(test['Patol'], test['Género'])
    separador = ' -----------------------------------------------------------------'
    result1 = pd.crosstab(test['Patol'], [dum1, test['Género']]).iloc[:,:8]
    result2 = pd.crosstab(test['Patol'], [dum1, test['Género']]).iloc[:,8:16]
    result3 = pd.crosstab(test['Patol'], [dum1, test['Género']]).iloc[:,16:24]
    result4 = pd.crosstab(test['Patol'], [dum1, test['Género']]).iloc[:,24:]
    result0
    result1
    result2
    result3
    result4

Period('2020-02', 'M')

SCA          91
PCR          42
PoliTMT     797
Otros      3630
Name: Patol, dtype: int64

Género,Masc,Feme
Patol,,
SCA,49,38
PCR,25,13
PoliTMT,362,291
Otros,1727,1818


Edad    [0, 5)      [5, 10)      [10, 15)      [15, 20)     
Género    Masc Feme    Masc Feme     Masc Feme     Masc Feme
Patol                                                       
SCA          0    2       0    1        1    1        2    2
PCR          2    0       0    0        1    0        0    0
PoliTMT      5    3      11   23       21   16       18   14
Otros       49   38      28   19       50   33       67   73

Edad    [20, 25)      [25, 30)      [30, 35)      [35, 40)     
Género      Masc Feme     Masc Feme     Masc Feme     Masc Feme
Patol                                                          
SCA            8    3        4    0        4    0        0    1
PCR            0    1        2    0        0    0        0    0
PoliTMT       52   34       34   24       45   26       16   15
Otros         92   64       86   70       72   77       64   63

Edad    [40, 45)      [45, 50)      [50, 55)      [55, 60)     
Género      Masc Feme     Masc Feme     Masc Feme     Masc Feme
Patol                                                          
SCA            1    1        3    1        2    4        1    2
PCR            0    0        0    0        2    0        2    4
PoliTMT       33   23       17   25       16   13       24   10
Otros         75   50       83   79       86   69      138   94

Edad    [60, 65)      [65, 70)      [70, 75)      [75, 80)      [80, 85)     
Género      Masc Feme     Masc Feme     Masc Feme     Masc Feme     Masc Feme
Patol                                                                        
SCA            2    2        4    4        6    1        3    4        3    4
PCR            3    2        4    0        0    1        3    3        3    1
PoliTMT       11   22        4    4        5    1        7   12        2    1
Otros        110  111       88  106      148  135      126  137      121  203

In [24]:
titulo = ' Sección N: TRASLADO SECUNDARIO (Desde un Establecimiento a Otro)'
texto = 'Es una tabla larga, por lo que se divide en varios espacios '

titulo
texto
#result

print(titulo, file=open('Informe.txt','a'))
print(texto, file=open('Informe.txt','a'))
print(result0, file=open('Informe.txt','a'))
print(separador,file=open('Informe.txt','a'))  #Separador
print(result1, file=open('Informe.txt','a'))
print(separador,file=open('Informe.txt','a'))  #Separador
print(result2, file=open('Informe.txt','a'))
print(separador,file=open('Informe.txt','a'))  #Separador
print(result3, file=open('Informe.txt','a'))
print(separador,file=open('Informe.txt','a'))  #Separador
print(result4, file=open('Informe.txt','a'))
print(separador,file=open('Informe.txt','a'))  #Separador
print(espacios, file=open('Informe.txt','a'))

' Sección N: TRASLADO SECUNDARIO (Desde un Establecimiento a Otro)'

'Es una tabla larga, por lo que se divide en varios espacios '

# Finalización
Fin informe REM8

In [25]:
print('Fin informe REM 8', file=open('Informe.txt','a'))


# Análisis posteriores extra

In [26]:
db1[db1['Motivo del Llamado']=='Otro'][['Submotivo del Llamado', 'Id']]

,Submotivo del Llamado,Id
241,Fallecido,137182
253,Fallecido,137165
260,Fallecido,137157
299,Caso social,137112
325,Fallecido,137088
346,Otros,137045
352,Fallecido,137034
382,Fallecido,136991
422,Fallecido,136919
449,Fallecido,136878


In [27]:
BD.Fecha.describe()

count                  110240
unique                  95928
top       2018-11-23 14:30:10
freq                      442
first     2018-01-15 08:45:15
last      2020-02-29 23:43:33
Name: Fecha, dtype: object